### Adjust Word Group Repeat With Youtube Link

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 1000  # 28 native word end index

# file extention
file_ext = "Fourgram"

In [4]:
def df_col_value_join_comma(df, df_columns_list):
    '''
    df_col_value_join_comma(df_test, ["video_id","start_time","end_time"])\n
    function used for selected column value join with comma in one row
    '''
    column_value_list = []
    for column in df_columns_list:
        list_var = df[f"{column}"].to_list()
        list_var_string = [str(x) for x in list_var] 
        list_var_join = ",".join(list_var_string)
        column_value_list.append(list_var_join)

    df_result = pd.DataFrame([[column_value_list[0],column_value_list[1],column_value_list[2]]], columns=df_columns_list)

    return df_result

In [5]:
path = f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Talk Time 2/Result/3-Adjust Word Group Repeat With Youtube Link"

Path(path).mkdir(parents=True, exist_ok=True)

In [6]:
path_folder_file = "/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/Turkish/Talk Time/Talk Time 2/Result/2-Find Word Group Youtube Link/Turkish_Fourgram_With_1000_Word_Selected_Count_Manuel.xlsx"
sheet = "Sheet1"  # Sheet1

In [7]:
df_word_group_time_loc = pd.read_excel(f"{path_folder_file}", sheet_name=f"{sheet}")
#df_word_group_time_loc = pd.read_excel(f"Twogram Threegram Youtube Link Selected_Manuel.xlsx")   
df_word_group_time_loc

,search_string,start_time,end_time,sentence,video_id,video_url,count
0,en son ne zaman,618,621,en son ne zaman gerçekten rahatsız oldun öneml...,eec5mG_90bk,https://www.youtube.com/watch?v=eec5mG_90bk&t=...,7
1,en son ne zaman,218,220,en son ne zaman bir bağış kumbarasına para att...,MyyM0EDVlCg,https://www.youtube.com/watch?v=MyyM0EDVlCg&t=...,7
2,en son ne zaman,8315,8318,sen en son ne zaman i stanbula geldin nenem,Bbaal2Neohk,https://www.youtube.com/watch?v=Bbaal2Neohk&t=...,7
3,en son ne zaman,625,628,şimdi şunu soruyorum en son ne zaman det gördünüz,85RxQHZkluE,https://www.youtube.com/watch?v=85RxQHZkluE&t=...,7
4,en son ne zaman,38,41,size desem ki en son ne zaman haksızlığa uğrad...,BhGXfFsxgsQ,https://www.youtube.com/watch?v=BhGXfFsxgsQ&t=38s,7
...,...,...,...,...,...,...,...
69,bu da ne böyle,3192,3195,bu da ne böyle,sMEBFgBFmy8,https://www.youtube.com/watch?v=sMEBFgBFmy8&t=...,4
70,bir sorun mu var,2646,2648,i yi akşamlar hayırdır bu saatte böyle gelmişs...,i_fb_jHCUE8,https://www.youtube.com/watch?v=i_fb_jHCUE8&t=...,4
71,bir sorun mu var,7198,7200,bir sorun mu var hanımefendi yanlış nik h,HhLIORa1NKg,https://www.youtube.com/watch?v=HhLIORa1NKg&t=...,4
72,bir sorun mu var,772,774,bir sorun mu var mert,qe8TjfIqZ8Y,https://www.youtube.com/watch?v=qe8TjfIqZ8Y&t=...,4


In [8]:
search_list = df_word_group_time_loc.iloc[:,0].unique()
len(search_list)

14

In [9]:
# other option 
# mUf7VNqChac =>  black screen
# 0_CDMstFg7M => 10sn
# bj1JRuyYeco => 20sn
# cElhIDdGz7M => screensaver
default_video_id = "Q-8I-uMUMYA"
df_link_default = pd.DataFrame(data=[["repeat",5,7,"repeat_again",f"{default_video_id}",f"https://www.youtube.com/watch?v={default_video_id}&t=0s"]], columns=["search_string","start_time","end_time","sentence","video_id","video_url"])
df_link_default

,search_string,start_time,end_time,sentence,video_id,video_url
0,repeat,5,7,repeat_again,Q-8I-uMUMYA,https://www.youtube.com/watch?v=Q-8I-uMUMYA&t=0s


In [10]:
## All word group result convert to join result in one row
#df_result_repeat = pd.DataFrame()
#for word_group in search_list:
#    # for repeat 1
#    df_word_group_search_repeat1 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group]
#    # for repeat 2
#    try:
#        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].sample(1)
#    except:
#        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].tail(1)
#    df_word_group_search_repeat2.reset_index(drop=True, inplace=True)   
#    # repeat block
#    for i in range(len(df_word_group_search_repeat1)):        
#        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat1.iloc[[i,]]], axis=0)
#
#    try:
#        df_link_default_var = df_link_default
#        word_time_diff_var = df_word_group_search_repeat2.loc[0 ,"end_time"] - df_word_group_search_repeat2.loc[0 ,"start_time"]
#        if word_time_diff_var < 4:
#            word_time_diff_var2 = 3
#        else:
#            word_time_diff_var2 = word_time_diff_var+1.0
#
#        df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + word_time_diff_var2
#    except:
#        pass
#
#    for j in range(3):
#        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat2], axis=0)
#        df_result_repeat = pd.concat([df_result_repeat,df_link_default_var], axis=0)
#
#df_result_repeat.reset_index(drop=True, inplace=True)
#df_result_repeat_join = df_col_value_join_comma(df_result_repeat, ["video_id","start_time","end_time"])
#
#df_result_repeat.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Talk_Time2.xlsx", index=False)
#df_result_repeat_join.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Talk_Time_Join2.xlsx", index=False)

In [11]:
#df_result_repeat

In [12]:
#(df_result_repeat["end_time"].sum()-df_result_repeat["start_time"].sum())/60

In [13]:
# Each word group result convert to join result in one row
df_all_join_result = pd.DataFrame()
seq_num = 0  # option
for word_group in search_list:
    df_result_repeat = pd.DataFrame()
    # for repeat 1
    df_word_group_search_repeat1 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group]
    # for repeat 2
    try:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].sample(1)
    except:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].tail(1)
    df_word_group_search_repeat2.reset_index(drop=True, inplace=True)   
    # repeat block
    for i in range(len(df_word_group_search_repeat1)):        
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat1.iloc[[i,]]], axis=0)

    try:
        df_link_default_var = df_link_default
        word_time_diff_var = df_word_group_search_repeat2.loc[0 ,"end_time"] - df_word_group_search_repeat2.loc[0 ,"start_time"]
        if word_time_diff_var < 4:
            word_time_diff_var2 = 3
        else:
            word_time_diff_var2 = word_time_diff_var+1.0

        df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + word_time_diff_var2
    except:
        pass

    for j in range(3):
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat2], axis=0)
        df_result_repeat = pd.concat([df_result_repeat,df_link_default_var], axis=0)
        df_result_repeat.reset_index(drop=True, inplace=True)

    df_result_repeat_var = df_col_value_join_comma(df_result_repeat, ["video_id","start_time","end_time"]) 
    df_result_repeat_var.loc[0,["search_string"]] = f"{word_group}"  # option for search string
    df_result_repeat_var.loc[0,["count"]] = df_result_repeat.loc[0,"count"] # option for count num     
    df_all_join_result = pd.concat([df_all_join_result,df_result_repeat_var], axis=0)

df_all_join_result.reset_index(drop=True, inplace=True)
df_all_join_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Youtube_Talk_Time_Join_{file_ext}.xlsx", index=False)


In [14]:
df_all_join_result

,video_id,start_time,end_time,search_string,count
0,"eec5mG_90bk,MyyM0EDVlCg,Bbaal2Neohk,85RxQHZklu...","618,218,8315,625,38,132,472,8315,5,8315,5,8315,5","621,220,8318,628,41,135,474,8318,8,8318,8,8318,8",en son ne zaman,7.0
1,"gQg3cyWahgc,_JDO9FKXZ88,lNwXQrpMfhM,259wx6vTLz...","7840,5802,1550,4352,833,5500,748,1550,5,1550,5...","7842,5804,1552,4354,835,5501,750,1552,8,1552,8...",bir şey var mı,7.0
2,"pLifvTWuvHg,gE85fd99w44,6Eypw0D5r78,SdycpaKESd...","7823,107,5968,2615,523,3488,1492,3488,5,3488,5...","7825,109,5970,2619,525,3490,1494,3490,8,3490,8...",bir şey mi oldu,7.0
3,"Aq7s33k_17A,Tm4hNlWWa_8,jhmfLDWagSk,6teAhXTWps...","72,650,370,2966,509,521,860,2966,5,2966,5,2966,5","74,653,372,2968,511,524,862,2968,8,2968,8,2968,8",bir şey daha var,7.0
4,"9GbYnL0tvcg,cUAQrHbbpZ8,cUAQrHbbpZ8,1iByM08Mvq...","1886,1696,2447,6004,1056,1098,6004,5,6004,5,60...","1889,1700,2450,6007,1059,1101,6007,8,6007,8,60...",seni görmek çok güzel,6.0
5,"UEUJAgUgP_Q,UGyGF6BzXTM,EC4gKcDZvug,l7xSPH97zv...","1688,694,1395,252,272,252,5,252,5,252,5","1691,697,1398,255,274,255,8,255,8,255,8",çok güzel değil mi,5.0
6,"NudrNCdBxAE,WluihGTzAzs,6O8r2LsU_cc,BPFVcPmoot...","463,161,92,565,221,565,5,565,5,565,5","465,163,95,567,223,567,8,567,8,567,8",o kadar da kötü,5.0
7,"gIgjKheyn3o,A3y7gcwGWCw,2urfZXpi6n0,UakI2BMHoJ...","2072,716,1298,3709,105,716,5,716,5,716,5","2074,719,1301,3711,107,719,8,719,8,719,8",bunu daha önce de,5.0
8,"LMHUJ60Fsho,NV1Izihxbks,oWyqKDfhtgo,EFy7Qr4mmZ...","78,106,510,475,4855,475,5,475,5,475,5","80,109,513,478,4857,478,8,478,8,478,8",bana yardım eder misin,5.0
9,"wa6FhuUXTRs,sXm5UoYzLZU,pUUoRExmU1E,pZTreTu_HF...","1474,330,413,145,1474,5,1474,5,1474,5","1477,332,416,147,1477,8,1477,8,1477,8",o kadar basit değil,4.0


#### Copy Move And Delete

In [15]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{word_end}_Word_Youtube_Talk_Time_Join_{file_ext}.xlsx")
output_file

['Turkish_1000_Word_Youtube_Talk_Time_Join_Fourgram.xlsx']

In [16]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [17]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

#### Temp

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
path_folder_file2 = "/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/Turkish/Talk Time/Talk Time 2/Result/2-Find Word Group Youtube Link/Turkish_Fourgram_With_1000_Word_Selected_Manuel.xlsx"
sheet2 = "Sheet1"  # Sheet1

In [ ]:
df = pd.read_excel(f"{path_folder_file2}", sheet_name=f"{sheet2}")
df

In [ ]:
df_count = df.groupby(["search_string"])[["search_string"]].count()
df_count.rename(columns={"search_string":"count"}, inplace=True)
df_count.sort_values(by="count", ascending=False, inplace=True)
df_count.reset_index(inplace=True)
df_count

In [ ]:
df_count_merge = pd.merge(df, df_count, how="inner",on= "search_string")
df_count_merge.sort_values(by=["count","search_string"], ascending=False, inplace=True)
df_count_merge.reset_index(drop=True, inplace=True)
df_count_merge

In [ ]:
df_count_merge["search_string"].nunique()

In [ ]:
df_count_merge[df_count_merge["count"] == 1]

In [ ]:
df_count_merge.to_excel("Turkish_Fourgram_With_1000_Word_Selected_Count_Manuel.xlsx", index=False)

In [ ]:
df_word_group_time_loc_manuel = pd.read_excel(f"Twogram Threegram Youtube Link Selected_Count_Manuel.xlsx")
df_word_group_time_loc_manuel.sort_values(by=["count","search_string"],inplace=True, ascending=False)
df_word_group_time_loc_manuel.reset_index(drop=True, inplace=True)
df_word_group_time_loc_manuel

In [ ]:
df_word_group_time_loc_manuel.to_excel("Twogram Threegram Youtube Link Selected_Manuel.xlsx", index=False)